In [12]:
!pip install -q py-feat
# !pip install -q facial-emotion-recognition
# !pip install -q fer
# !pip install -q deepface
!pip install -q mtcnn

     |████████████████████████████████| 6.0MB 5.1MB/s 
     |████████████████████████████████| 3.3MB 34.1MB/s 
     |████████████████████████████████| 57.4MB 95kB/s 
     |████████████████████████████████| 4.9MB 33.7MB/s 


In [14]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from feat import Detector
import glob
import math
import dlib
import joblib
import cv2
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
from collections import Counter
from mtcnn import MTCNN

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [16]:
################# py-feat
face_model = "retinaface"
landmark_model = "MobileNet"
au_model = "svm"
emotion_model = "resmasknet" #resmasknet,fer, svm, rf
detector = Detector(face_model = face_model, landmark_model = landmark_model, au_model = au_model, emotion_model = emotion_model)

detector_MTCNN = MTCNN()

Loading Face Detection model:  retinaface



Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib



Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_scalar_aus.joblib



Loading Face Landmark model:  MobileNet
Loading au model:  svm
Loading emotion model:  resmasknet


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
ls 'drive/MyDrive/file2/vdo/Anongrat/01_blackping.mp4'

drive/MyDrive/file2/vdo/Anongrat/01_blackping.mp4


In [11]:
ls 'drive/MyDrive/file2/files/01_blackping_P1.csv'

drive/MyDrive/file2/files/01_blackping_P1.csv


In [ ]:
from shutil import copyfile, copy
filepath = 'drive/MyDrive/file2/files/01_blackping_P1.csv'
vdopath = 'drive/MyDrive/file2/vdo/Anongrat/01_blackping.mp4'
df = pd.read_csv(filepath)
dictT = {'Neutral':'neutral', 'มีความสุข':'happy', 'ผ่อนคลาย':'neutral', 'เครียด':'angry', 'เศร้า':'sad',
         'มีความสุช':'happy', 'โกรธ':'angry', 'มีความสุุข':'happy', 'มีความสูข':'happy', 'ผ่่อนคลาย':'neutral',
         'ตื่นเต้น':'happy', 'ประหลาดใจ':'surprise', 'จริงจัง':'angry', 'neutral ': 'neutral', 'ผ่อ่นคลาย':'neutral',
         'มีตวามสุข':'happy', 'โกรธ ':'angry'}
df['label2'] = df['label2'].replace(dictT)
print([pd.unique(df['label2'])])

for i in tqdm(range(df.shape[0])):
  for j in range(df['start_frame'][i], df['end_frame'][i]):
    cap = cv2.VideoCapture(vdopath)
    cap.set(cv2.CAP_PROP_POS_FRAMES, j)
    success, image = cap.read()
    faceimg = detector.detect_faces(image)
    pos = faceimg[0][:4] # [FaceRectX, FaceRectY, FaceRectWidth, FaceRectHeight]
    cropimg =image[int(pos[1]):int(pos[3]),int(pos[0]):int(pos[2])] #[x, y, width, height]  
    dim = (224, 224)
    resized = cv2.resize(cropimg, dim)
    # cv2_imshow(resized)
    filename = 'lab.png'
    cv2.imwrite(filename, resized)
    emotion = df['label2'][i]
    dst = 'lab'+str(j)+'.png'
    terminal_path = "drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/lab/{0}/{1}".format(emotion, dst)
    copyfile('/content/lab.png', terminal_path)

[array(['neutral', 'happy'], dtype=object)]
